# Lab 2: Parallelization and Evaluator-Optimizer Pattern

This notebook implements the **Evaluator-Optimizer Pattern** with **Parallelization**:

1. **Evaluator**: Gathers API keys and prepares model configurations
2. **Parallel Execution**: All models run simultaneously using async/await
3. **Aggregator**: Collects and formats all outputs for evaluation
4. **Final Evaluator**: Judge model ranks all responses from best to worst

## Pattern Flow

```
Evaluator (API Keys & Configs) 
    ↓
Parallel API Calls (All models run simultaneously)
    ↓
Aggregator (Collect & Format outputs)
    ↓
Final Evaluator (Judge ranks all outputs)
```

In [ ]:
# Start with imports - ask ChatGPT to explain any package that you don't know

import os
import json
import asyncio
import random
from datetime import datetime, timedelta
from typing import Dict, List, Tuple, Any
from dotenv import load_dotenv
from openai import OpenAI
from anthropic import Anthropic
from IPython.display import Markdown, display

In [ ]:
# Always remember to do this!
load_dotenv(override=True)

In [ ]:
# Print the key prefixes to help with any debugging

openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set (and this is optional)")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:2]}")
else:
    print("Google API Key not set (and this is optional)")

if deepseek_api_key:
    print(f"DeepSeek API Key exists and begins {deepseek_api_key[:3]}")
else:
    print("DeepSeek API Key not set (and this is optional)")

if groq_api_key:
    print(f"Groq API Key exists and begins {groq_api_key[:4]}")
else:
    print("Groq API Key not set (and this is optional)")

## Step 1: Generate Question

First, we'll generate a challenging question to ask all the models.

In [ ]:
request = "Please come up with a challenging, nuanced question that I can ask a number of LLMs to evaluate their intelligence. "
request += "Answer only with the question, no explanation."
messages = [{"role": "user", "content": request}]

In [ ]:
openai = OpenAI()
response = openai.chat.completions.create(
    model="gpt-5-mini",
    messages=messages,
)
question = response.choices[0].message.content
print(question)

In [ ]:
# Prepare the question for all models
competitors = []
answers = []
messages = [{"role": "user", "content": question}]

## Step 2: Evaluator - Prepare API Keys and Configurations

The **Evaluator** gathers API keys and prepares configurations for all models.

In [ ]:
# ==========================================
# Evaluator: Gathers API keys and prepares model configurations
# ==========================================
# This function prepares all model configurations ready for parallel execution

def evaluator_prepare_configs():
    """
    Evaluator: Gathers API keys and prepares configurations for all models.
    Returns a list of model configurations ready for parallel execution.
    """
    configs = []
    
    # Model 1: OpenAI
    configs.append({
        "model_name": "gpt-5-nano",
        "provider": "openai",
        "client": OpenAI(),
        "call_type": "chat.completions",
        "extra_params": {}
    })
    
    # Model 2: Anthropic
    configs.append({
        "model_name": "claude-sonnet-4-5",
        "provider": "anthropic",
        "client": Anthropic(),
        "call_type": "messages.create",
        "extra_params": {"max_tokens": 1000}
    })
    
    # Model 3: Gemini
    configs.append({
        "model_name": "gemini-2.5-flash",
        "provider": "gemini",
        "client": OpenAI(api_key=google_api_key, base_url="https://generativelanguage.googleapis.com/v1beta/openai/"),
        "call_type": "chat.completions",
        "extra_params": {}
    })
    
    # Model 4: DeepSeek
    configs.append({
        "model_name": "deepseek/deepseek-r1-0528:free",
        "provider": "deepseek",
        "client": OpenAI(api_key=deepseek_api_key, base_url="https://openrouter.ai/api/v1"),
        "call_type": "chat.completions",
        "extra_params": {}
    })
    
    # Model 5: Groq
    configs.append({
        "model_name": "openai/gpt-oss-120b",
        "provider": "groq",
        "client": OpenAI(api_key=groq_api_key, base_url="https://api.groq.com/openai/v1"),
        "call_type": "chat.completions",
        "extra_params": {}
    })
    
    # Model 6: Ollama (if available)
    configs.append({
        "model_name": "llama3.2",
        "provider": "ollama",
        "client": OpenAI(base_url='http://localhost:11434/v1', api_key='ollama'),
        "call_type": "chat.completions",
        "extra_params": {}
    })
    
    print(f"✅ Evaluator prepared {len(configs)} model configurations")
    return configs

# Prepare all configurations
model_configs = evaluator_prepare_configs()

## Step 3: Parallel Execution - Call All Models Simultaneously

All models are called **in parallel** using async/await, making the process much faster than sequential calls.

In [ ]:
# ==========================================
# Async function to call a single model
# ==========================================

async def call_model_async(config: Dict[str, Any], messages: List[Dict]) -> Tuple[str, str]:
    """
    Call a single model asynchronously. Returns (model_name, answer) or (model_name, error_message).
    """
    model_name = config["model_name"]
    provider = config["provider"]
    client = config["client"]
    call_type = config["call_type"]
    extra_params = config["extra_params"]
    
    try:
        if provider == "anthropic":
            # Anthropic uses a different API structure
            response = await asyncio.to_thread(
                client.messages.create,
                model=model_name,
                messages=messages,
                **extra_params
            )
            answer = response.content[0].text
        else:
            # OpenAI-compatible APIs
            response = await asyncio.to_thread(
                client.chat.completions.create,
                model=model_name,
                messages=messages,
                **extra_params
            )
            answer = response.choices[0].message.content
        
        print(f"✅ {model_name} completed")
        return model_name, answer
        
    except Exception as e:
        error_msg = f"Error calling {model_name}: {str(e)}"
        print(f"❌ {error_msg}")
        return model_name, error_msg

# ==========================================
# Parallel execution function
# ==========================================

def format_bytes(size: int) -> str:
    """Format bytes into a human-readable string (B, KB, MB)."""
    for unit in ['B', 'KB', 'MB']:
        if size < 1024.0:
            return f"{size:.2f} {unit}"
        size /= 1024.0
    return f"{size:.2f} GB"

async def execute_models_in_parallel(configs: List[Dict[str, Any]], messages: List[Dict]) -> Tuple[List[str], List[str]]:
    # Overall execution start
    print(f"\n🚀 Starting parallel execution of {len(configs)} models...\n")
    
    # Track data for the table
    table_rows = []
    competitors = []
    answers = []
    
    async def call_with_metrics(config):
        model_name = config.get("model_name", "Unknown")
        start_time = datetime.now()
        
        try:
            # Assumes call_model_async returns (model_name, answer)
            _, answer = await call_model_async(config, messages)
            end_time = datetime.now()
            
            # Check for error strings inside the success path
            if isinstance(answer, str) and answer.startswith("Error"):
                status = "❌ Error"
                out_size = 0
            else:
                status = "✅ Success"
                out_size = len(str(answer).encode('utf-8'))
                
        except Exception as e:
            end_time = datetime.now()
            status = "❌ Error"
            answer = str(e)
            out_size = 0

        # Calculate duration
        duration = end_time - start_time
        total_seconds = int(duration.total_seconds())
        mm, ss = divmod(total_seconds, 60)
        hh, mm = divmod(mm, 60)
        dur_str = f"{hh:02d}:{mm:02d}:{ss:02d}" if hh > 0 else f"{mm:02d}:{ss:02d}"

        # Store metrics for table
        table_rows.append({
            "model": model_name,
            "status": status,
            "start": start_time.strftime("%H:%M:%S"),
            "end": end_time.strftime("%H:%M:%S"),
            "duration": dur_str,
            "size": format_bytes(out_size)
        })
        
        return model_name, answer, status

    # Run tasks in parallel
    tasks = [call_with_metrics(config) for config in configs]
    results = await asyncio.gather(*tasks)

    # Process final lists
    for model_name, answer, status in results:
        if status == "✅ Success":
            competitors.append(model_name)
            answers.append(answer)

    # Print Tabular Output
    header = f"{'Model':<20} {'Status':<10} {'Start':<10} {'End':<10} {'Duration':<10} {'Size':<12}"
    print(header)
    print("-" * len(header))
    for row in table_rows:
        print(f"{row['model']:<20} {row['status']:<10} {row['start']:<10} {row['end']:<10} {row['duration']:<10} {row['size']:<12}")
    
    print(f"\n✅ Completed. {len(competitors)}/{len(configs)} models successful.")
    return competitors, answers

async def mock_execute_models_in_parallel(configs: List[Dict[str, Any]]) -> Tuple[List[str], List[str]]:
    """
    Mocks parallel API calls to display timing and size metrics in a table.
    No actual API calls are made.
    """
    print(f"\n🚀 Starting MOCK execution of {len(configs)} models...\n")
    
    table_rows = []
    competitors = []
    answers = []

    async def mock_api_call(config):
        model_name = config.get("model_name", "Unknown-Model")
        start_time = datetime.now()
        
        # Simulate varying network latency (0.5 to 2.5 seconds)
        await asyncio.sleep(random.uniform(0.5, 2.5))
        
        # Randomly decide if this mock call "fails" (10% chance)
        is_success = random.random() > 0.1
        
        if is_success:
            status = "✅ Success"
            # Mock a response string of random length
            mock_answer = "Mock response data " * random.randint(5, 500)
            out_size = len(mock_answer.encode('utf-8'))
        else:
            status = "❌ Error"
            mock_answer = "Error: Mocked API failure"
            out_size = 0
            
        end_time = datetime.now()
        
        # Calculate duration in mm:ss or hh:mm:ss
        duration = end_time - start_time
        total_seconds = int(duration.total_seconds())
        mm, ss = divmod(total_seconds, 60)
        hh, mm = divmod(mm, 60)
        dur_str = f"{hh:02d}:{mm:02d}:{ss:02d}" if hh > 0 else f"{mm:02d}:{ss:02d}"

        # Record metrics for the final table
        metrics = {
            "model": model_name,
            "status": status,
            "start": start_time.strftime("%H:%M:%S"),
            "end": end_time.strftime("%H:%M:%S"),
            "duration": dur_str,
            "size": format_bytes(out_size)
        }
        
        return model_name, mock_answer, status, metrics

    # Execute mock tasks in parallel
    tasks = [mock_api_call(config) for config in configs]
    results = await asyncio.gather(*tasks)

    # Prepare table headers
    header = f"{'Model':<20} {'Status':<10} {'Start':<10} {'End':<10} {'Duration':<10} {'Size':<12}"
    print(header)
    print("-" * len(header))

    # Output rows and collect final success data
    for model_name, answer, status, row in results:
        print(f"{row['model']:<20} {row['status']:<10} {row['start']:<10} {row['end']:<10} {row['duration']:<10} {row['size']:<12}")
        if status == "✅ Success":
            competitors.append(model_name)
            answers.append(answer)

    print(f"\n✅ Completed. {len(competitors)}/{len(configs)} models simulated successfully.")
    return competitors, answers


In [ ]:

# --- Mock API Calls ---
competitors, answers = await mock_execute_models_in_parallel(model_configs)


In [ ]:
# Execute all models in parallel using the async functions
competitors, answers = await execute_models_in_parallel(model_configs, messages)
        


In [ ]:
# Optionally - Display the answers
# for model_name, answer in zip(competitors, answers):
#     display(Markdown(f"### {model_name}\n\n{answer}"))


## Step 4: Aggregator - Collect and Format Outputs

The **Aggregator** collects all model outputs and formats them for the final evaluator.

In [ ]:
# ==========================================
# Aggregator: Collect outputs and format for evaluation
# ==========================================
# The Aggregator collects all model outputs and prepares them
# for the final Evaluator (judge) that will rank the responses

def aggregator_format_outputs(competitors: List[str], answers: List[str]) -> str:
    """
    Aggregator: Collects all model outputs and formats them for evaluation.
    Returns a formatted string ready for the judge/evaluator.
    """
    together = ""
    for index, answer in enumerate(answers):
        together += f"# Response from competitor {index+1}\n\n"
        together += answer + "\n\n"
    return together

# Use the aggregator to format all outputs
together = aggregator_format_outputs(competitors, answers)
print(f"✅ Aggregator collected and formatted {len(competitors)} model responses")

## Step 5: Final Evaluator - Judge and Rank All Outputs

The **Final Evaluator** (Judge) evaluates all aggregated responses and ranks them from best to worst.

In [ ]:
# ==========================================
# Final Evaluator: Judge and Rank All Outputs
# ==========================================
# The final Evaluator (Judge) model evaluates all aggregated responses
# and ranks them from best to worst

judge = f"""You are judging a competition between {len(competitors)} competitors.
Each model has been given this question:

{question}

Your job is to evaluate each response for clarity and strength of argument, and rank them in order of best to worst.
Respond with JSON, and only JSON, with the following format:
{{"results": ["best competitor number", "second best competitor number", "third best competitor number", ...]}}

Here are the responses from each competitor:

{together}

Now respond with the JSON with the ranked order of the competitors, nothing else. Do not include markdown formatting or code blocks."""

judge_messages = [{"role": "user", "content": judge}]

In [ ]:
# ==========================================
# Final Evaluator Call: Judge Ranks All Outputs
# ==========================================
# The Evaluator (Judge) model evaluates and ranks all model responses

openai = OpenAI()
response = openai.chat.completions.create(
    model="gpt-5-mini",
    messages=judge_messages,
)
results = response.choices[0].message.content
print("✅ Final Evaluator (Judge) completed ranking:")
print(results)

In [ ]:
# Parse and display the final rankings
results_dict = json.loads(results)
ranks = results_dict["results"]
print("\n" + "="*50)
print("FINAL RANKINGS")
print("="*50)
for index, result in enumerate(ranks):
    competitor = competitors[int(result)-1]
    print(f"Rank {index+1}: {competitor}")